In [0]:

# Load OpenAI API Key from Secret Scope

openai_api_key = dbutils.secrets.get(scope="adls_scope", key="open-ai-api-key")
print("API key loaded successfully.")


In [0]:
# Read Parameters from Job Config
store_id = spark.conf.get("spark.store_id", "")
category = spark.conf.get("spark.category", "")

print(f"Parameters → store_id: '{store_id}', category: '{category}'")

In [0]:
# Build Dynamic SQL Based on Parameters

base_query = """
    SELECT store_id, category, product_name, SUM(sale_amount) AS total_sales
    FROM vw_sales_transaction_summary
"""

where_clauses = []
if store_id:
    where_clauses.append(f"store_id = {int(store_id)}")
if category:
    where_clauses.append(f"category = '{category}'")

if where_clauses:
    base_query += "WHERE " + " AND ".join(where_clauses) + "\n"

base_query += "GROUP BY store_id, category, product_name\nORDER BY store_id, category"

gold_df = spark.sql(base_query)
display(gold_df)

In [0]:

%pip install tabulate
%pip install requests


In [0]:

# Convert Data to Tabular Format
import pandas as pd
import requests
import json
from tabulate import tabulate

gold_pd_df = gold_df.toPandas()
table_summary = tabulate(gold_pd_df, headers="keys", tablefmt="pipe", showindex=False)

# Generate GPT Prompt
prompt = f"""
You are a helpful AI assistant. Summarize the key sales trends from the following table.
Focus on high-performing products by store and category.

{table_summary}
"""

# Send Request to OpenAI
api_url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_api_key}"
}

body = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role": "system", "content": "You are a data analyst assistant."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.3
}

response = requests.post(api_url, headers=headers, data=json.dumps(body))

# Show Summary
if response.status_code == 200:
    summary = response.json()["choices"][0]["message"]["content"]
    print("\nGPT-Generated Summary:\n")
    print(summary)
else:
    print("Error:", response.status_code, response.text)


In [0]:
# Create the schema if it does not exist
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

In [0]:
from pyspark.sql.functions import lit, current_timestamp

# Save GPT result into a table for dashboard
summary_df = spark.createDataFrame([(summary,)], ["gpt_summary"]) \
                  .withColumn("timestamp", current_timestamp())

summary_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("gold.gpt_sales_summary")
